# Summer School

## Teil 4: Named entity recognition

In [ ]:
import pandas as pd
import os
import spacy
from spacy import displacy
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from string import punctuation
from collections import Counter

In [ ]:
df = pd.read_csv("otra_alemania_content.csv", encoding="utf-8")
df

In [ ]:
#Code für den Download eines der Sprachmodelle: 
#!python -m spacy download de_core_news_lg

#Laden eines Trained Model for German
nlp = spacy.load('de_core_news_sm')
print(nlp.pipe_names)

#### Umwandlung der einzelnen Wörter in spacy-doc-objects: 

Beispiel: 

In [ ]:
sentence = 'Dies ist ein "Test", oder?'
doc = nlp(sentence)

In [ ]:
for token in doc:
    print(token.text, token.pos_)

In [ ]:
# Wähle ein spezifisches Token (z.B. das erste Token)
test_token = doc[0]  # "Das"

# Ausgabe aller verfügbaren Attribute des Tokens
print(f"Token: {test_token.text}")
print(f"Index: {test_token.i}")
print(f"Text: {test_token.text}")
print(f"Lemma: {test_token.lemma_}")
print(f"Part-of-Speech: {test_token.pos_}")
print(f"Tag: {test_token.tag_}")
print(f"Dependency: {test_token.dep_}")
print(f"Shape: {test_token.shape_}")
print(f"Is Alpha: {test_token.is_alpha}")
print(f"Is Stop: {test_token.is_stop}")
print(f"Vector: {test_token.vector}")  # Nur verfügbar, wenn das Modell Vektoren unterstützt

# Ausgabe der benutzerdefinierten Attribute (falls vorhanden)
if hasattr(test_token._, 'my_custom_attr'):
    print(f"Benutzerdefiniertes Attribut: {test_token._.my_custom_attr}")

Funktion zur Erstellung der doc-Objekte für die Texte: 

In [ ]:
def process_text(text):
    text = str(text)
    return nlp(text)

In [ ]:
df['doc'] = df['text'].apply(process_text)

In [ ]:
df

### Dependency parsing

In [ ]:
doc = df['doc'][2]
sentences = list(doc.sents)
sentence = sentences[0]
print(sentence)
displacy.render(sentence, style="dep", jupyter=True)

## Named entity recogintion

Anzeige der im heruntergeladenen Sprachmodell enthaltenen Entitäts-Typen: 

In [ ]:
labels = nlp.get_pipe("ner").labels

for label in labels:
    print(label + ' : ' + spacy.explain(label))

Extraktion der einzelnen named-entity-Typen:

In [ ]:
# Funktion zur Extraktion der einzelnen named entities:
def extract_named_entities(doc):
    return [ent.label_ for ent in doc.ents]

#Erstellen einer neuen Spalte für die Typen: 
df['Named_Entities'] = df['doc'].apply(extract_named_entities)
df

Ergänzung der einzelnen named entities: 

In [ ]:
def extract_named_entities(doc):
    return [ent for ent in doc.ents]

df['NE_Words'] = df['doc'].apply(extract_named_entities)
df

#### Beispiel anhand des Textes aus einer gewählten Zeile: 

In [ ]:
doc = df['doc'][1]
colors = {'PER': '#99ff99'}  # Helle Rotfarbe für Personen
options = {'colors': colors}
displacy.render(doc, style='ent', jupyter=True, options=options)

### Extraktion eines gewählten Entitätentyps am Beispiel von Personen: 

In [ ]:
persons = []

def get_persons(doc):
    for ent in doc.ents:
        if ent.label_ == "PER":
           persons.append(ent)
    
df['doc'].apply(get_persons)

print(persons)

In [ ]:
print(len(persons))

In [ ]:
print(persons[0])

In [ ]:
print(type(persons[0]))

In [ ]:
entities = [ent.text for ent in persons]
entity_counts = Counter(entities)

In [ ]:
persons_list = list(entity_counts.items())
pers = pd.DataFrame(persons_list, columns=['Entity', 'Count'])

In [ ]:
pers

In [ ]:
df_sorted = pers.sort_values(by='Count', ascending=False)
df_sorted.head(30)